In [1]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from scipy.signal import hilbert
from scipy.signal import find_peaks
import os

Datos Lunares

In [11]:

Directorio = 'data/lunar/test/data/'

# Listar todas las carpetas dentro del directorio
carpetas = [carpeta for carpeta in os.listdir(Directorio) if os.path.isdir(os.path.join(Directorio, carpeta))]

SismosLuna = pd.DataFrame()

# Iterar sobre cada carpeta
for carpeta in carpetas:
    DirectorioF = os.path.join(Directorio, carpeta)  # Ruta completa de la carpeta
    print(f'Procesando carpeta: {DirectorioF}')

    archivos = os.listdir(DirectorioF)
    
    # Filtrar archivos que sean CSV y mseed
    archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
    archivos_mseed = [archivo for archivo in archivos if archivo.endswith('.mseed')]
    
    # Verificar si ambas listas tienen la misma longitud
    if len(archivos_csv) != len(archivos_mseed):
        print(f'Error: Número de archivos CSV ({len(archivos_csv)}) y mseed ({len(archivos_mseed)}) no coincide en {DirectorioF}')
        continue  # Saltar esta carpeta si los archivos no coinciden
    
    # Iterar sobre los archivos CSV y mseed al mismo tiempo
    for archivo_csv, archivo_mseed in zip(archivos_csv, archivos_mseed):
        ruta_csv = os.path.join(DirectorioF, archivo_csv)  # Ruta completa al archivo CSV
        ruta_mseed = os.path.join(DirectorioF, archivo_mseed)  # Ruta completa al archivo mseed
        
        # Leer archivo CSV
        DataTraining = pd.read_csv(ruta_csv)
        print(f'Leyendo archivo CSV: {ruta_csv}')
        
        # Leer archivo mseed
        st = read(ruta_mseed)
        print(f'Leyendo archivo mseed: {ruta_mseed}')


        # This is how you get the data and the time, which is in seconds
        tr = st.traces[0].copy()
        tr_times = tr.times()
        tr_data = tr.data


        print("Reducción de ruido por filtro high-pass, band-pass")
        # Definir la frecuencia de corte para el filtro high-pass
        cutoff_freq = 3  # Ajusta este valor según sea necesario

        # Crear una copia del objeto Stream
        st_filt_hp = st.copy()

        # Aplicar un filtro high-pass
        st_filt_hp.filter('highpass', freq=cutoff_freq)


        # Set the minimum frequency
        minfreq = 0.5
        maxfreq = 1

        # Going to create a separate trace for the filter data
        st_filt = st_filt_hp.copy()
        st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
        tr_filt = st_filt.traces[0].copy()
        tr_times_filt = tr_filt.times()
        tr_data_filt = tr_filt.data
        print("Finaliza reducción")


        print("Ejecutando la transformada de hilbert")
        analytical_signal = hilbert(tr_data_filt)
        amplitude_envelope = np.abs(analytical_signal)
        print("Transformada de hilbert finalizada")


        print("Iniciando detección de sismos")
        # Detectar picos en la energía
        # Ajustar el umbral de acuerdo a tus datos
        min_distance = 40000
        threshold = np.percentile(amplitude_envelope, 99.5)  # Ajustar percentil si es necesario
        peaks, properties = find_peaks(amplitude_envelope, height=threshold, distance=min_distance)

        # Definir ventana para acumular energía alrededor de cada pico
        window_size = 5000  # Puedes ajustar este tamaño según tus datos

        # Calcular la energía acumulada en la vecindad de cada pico
        accumulated_energy = []
        for peak in peaks:
            left = max(0, peak - window_size)
            right = min(len(amplitude_envelope), peak + window_size)
            energy_sum = np.sum(amplitude_envelope[left:right])
            accumulated_energy.append(energy_sum)

        # Seleccionar el pico con mayor energía acumulada
        max_energy_peak_idx = np.argmax(accumulated_energy)
        best_peak = peaks[max_energy_peak_idx]
        print("Detección de sismos finalizada")

        print("Exportando los valores del rango del sismo a una carpeta")
        # Encontrar los índices donde la envolvente supera el umbral
        above_threshold = amplitude_envelope > threshold
        RangoSismo = DataTraining.iloc[above_threshold]


        output_dir = 'output/data_rango_sismo/'  # Directorio donde se guardarán los archivos

        # Crear el directorio de salida si no existe
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Definir un nombre único para el archivo basado en la carpeta y archivo original
        nombre_salida = f'DataRangoSismo_{carpeta}_{archivo_csv[:-4]}.csv'
        ruta_salida = os.path.join(output_dir, nombre_salida)
        
        # Guardar el archivo CSV
        RangoSismo.to_csv(ruta_salida, index=False)
        print(f'Archivo guardado en: {ruta_salida}')

        PicoSismo = DataTraining.iloc[[best_peak]]

        # Agregar las filas al DataFrame consolidado usando pd.concat
        SismosLuna = pd.concat([SismosLuna, PicoSismo], ignore_index=True)

SismosLuna.to_csv('Sismos_Luna.csv', index=False)






Procesando carpeta: data/lunar/test/data/S12_GradeB
Leyendo archivo CSV: data/lunar/test/data/S12_GradeB\xa.s12.00.mhz.1969-12-16HR00_evid00006.csv
Leyendo archivo mseed: data/lunar/test/data/S12_GradeB\xa.s12.00.mhz.1969-12-16HR00_evid00006.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo/DataRangoSismo_S12_GradeB_xa.s12.00.mhz.1969-12-16HR00_evid00006.csv
Leyendo archivo CSV: data/lunar/test/data/S12_GradeB\xa.s12.00.mhz.1970-01-09HR00_evid00007.csv
Leyendo archivo mseed: data/lunar/test/data/S12_GradeB\xa.s12.00.mhz.1970-01-09HR00_evid00007.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detecció

Datos de Marte

In [14]:
Directorio = 'data/mars/test/data/'




archivos = os.listdir(Directorio)
SismosMarte = pd.DataFrame()
# Filtrar archivos que sean CSV y mseed
archivos_csv = [archivo for archivo in archivos if archivo.endswith('.csv')]
archivos_mseed = [archivo for archivo in archivos if archivo.endswith('.mseed')]

    
    # Iterar sobre los archivos CSV y mseed al mismo tiempo
for archivo_csv, archivo_mseed in zip(archivos_csv, archivos_mseed):
    ruta_csv = os.path.join(Directorio, archivo_csv)  # Ruta completa al archivo CSV
    ruta_mseed = os.path.join(Directorio, archivo_mseed)  # Ruta completa al archivo mseed
        
    # Leer archivo CSV
    DataTraining = pd.read_csv(ruta_csv)
    print(f'Leyendo archivo CSV: {ruta_csv}')
        
    # Leer archivo mseed
    st = read(ruta_mseed)
    print(f'Leyendo archivo mseed: {ruta_mseed}')


    # This is how you get the data and the time, which is in seconds
    tr = st.traces[0].copy()
    tr_times = tr.times()
    tr_data = tr.data


    print("Reducción de ruido por filtro high-pass, band-pass")
    # Definir la frecuencia de corte para el filtro high-pass
    cutoff_freq = 3  # Ajusta este valor según sea necesario

    # Crear una copia del objeto Stream
    st_filt_hp = st.copy()

    # Aplicar un filtro high-pass
    st_filt_hp.filter('highpass', freq=cutoff_freq)


    # Set the minimum frequency
    minfreq = 0.5
    maxfreq = 1

    # Going to create a separate trace for the filter data
    st_filt = st_filt_hp.copy()
    st_filt.filter('bandpass',freqmin=minfreq,freqmax=maxfreq)
    tr_filt = st_filt.traces[0].copy()
    tr_times_filt = tr_filt.times()
    tr_data_filt = tr_filt.data
    print("Finaliza reducción")


    print("Ejecutando la transformada de hilbert")
    analytical_signal = hilbert(tr_data_filt)
    amplitude_envelope = np.abs(analytical_signal)
    print("Transformada de hilbert finalizada")


    print("Iniciando detección de sismos")
    # Detectar picos en la energía
    # Ajustar el umbral de acuerdo a tus datos
    min_distance = 40000
    threshold = np.percentile(amplitude_envelope, 99.5)  # Ajustar percentil si es necesario
    peaks, properties = find_peaks(amplitude_envelope, height=threshold, distance=min_distance)

    # Definir ventana para acumular energía alrededor de cada pico
    window_size = 5000  # Puedes ajustar este tamaño según tus datos

    # Calcular la energía acumulada en la vecindad de cada pico
    accumulated_energy = []
    for peak in peaks:
        left = max(0, peak - window_size)
        right = min(len(amplitude_envelope), peak + window_size)
        energy_sum = np.sum(amplitude_envelope[left:right])
        accumulated_energy.append(energy_sum)

    # Seleccionar el pico con mayor energía acumulada
    max_energy_peak_idx = np.argmax(accumulated_energy)
    best_peak = peaks[max_energy_peak_idx]
    print("Detección de sismos finalizada")

    print("Exportando los valores del rango del sismo a una carpeta")
    # Encontrar los índices donde la envolvente supera el umbral
    above_threshold = amplitude_envelope > threshold
    RangoSismoMarte = DataTraining.iloc[above_threshold]


    output_dir = 'output/data_rango_sismo_Marte/'  # Directorio donde se guardarán los archivos

    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Definir un nombre único para el archivo basado en la carpeta y archivo original
    nombre_salida = f'DataRangoSismo_{archivo_csv[:-4]}.csv'
    ruta_salida = os.path.join(output_dir, nombre_salida)
        
    # Guardar el archivo CSV
    RangoSismoMarte.to_csv(ruta_salida, index=False)
    print(f'Archivo guardado en: {ruta_salida}')

    PicoSismo = DataTraining.iloc[[best_peak]]

    # Agregar las filas al DataFrame consolidado usando pd.concat
    SismosMarte = pd.concat([SismosMarte, PicoSismo], ignore_index=True)

SismosMarte.to_csv('Sismos_Marte.csv', index=False)

Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en: output/data_rango_sismo_Marte/DataRangoSismo_XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv
Leyendo archivo CSV: data/mars/test/data/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.csv
Leyendo archivo mseed: data/mars/test/data/XB.ELYSE.02.BHV.2019-07-26HR12_evid0033.mseed
Reducción de ruido por filtro high-pass, band-pass
Finaliza reducción
Ejecutando la transformada de hilbert
Transformada de hilbert finalizada
Iniciando detección de sismos
Detección de sismos finalizada
Exportando los valores del rango del sismo a una carpeta
Archivo guardado en:

In [7]:
best_peak

38779